# MVP-V: 스테레오 깊이 추정

이 노트북은 스테레오 이미지를 사용하여 깊이 맵을 추정합니다.

## 목표
- 스테레오 이미지 기반 깊이 추정 (OpenCV StereoBM/StereoSGBM)
- 깊이 맵 시각화
- 깊이 + YOLO 박스 → 3D 위치 추정

## 참고
- KITTI는 스테레오 이미지를 제공하므로 절대 스케일이 보장됨 (monocular depth 대비 우위)

In [ ]:
# Cell 1: 라이브러리 설치 및 임포트
import numpy as np
import cv2
import matplotlib.pyplot as plt
from pathlib import Path
from mpl_toolkits.mplot3d import Axes3D

# 경로 설정
project_root = Path.cwd().parent.parent
dataset_root = project_root / "dataset" / "kitti"
print(f"프로젝트 루트: {project_root}")

In [ ]:
# Cell 2: 칼리브레이션 정보 로딩
def load_calibration(calib_path):
    """KITTI 칼리브레이션 파일 로딩"""
    calib = {}
    if not calib_path.exists():
        return calib
    
    with open(calib_path, 'r') as f:
        for line in f:
            if ':' in line:
                key, value = line.strip().split(':', 1)
                calib[key.strip()] = np.array([float(x) for x in value.strip().split()]).reshape(3, -1)
    
    return calib

# 칼리브레이션 로드 (예시 경로)
training_calib = project_root / "dataset" / "training" / "calib"
if training_calib.exists():
    sample_calib = list(training_calib.glob("*.txt"))[0] if list(training_calib.glob("*.txt")) else None
    if sample_calib:
        calib = load_calibration(sample_calib)
        print("칼리브레이션 로드 완료")
        if 'P2' in calib:
            print(f"P2 (왼쪽 카메라):\n{calib['P2']}")
        if 'P3' in calib:
            print(f"P3 (오른쪽 카메라):\n{calib['P3']}")
else:
    print("칼리브레이션 파일을 찾을 수 없습니다.")

In [ ]:
# Cell 3: 스테레오 매칭기 설정
def create_stereo_matcher(method='SGBM'):
    """
    스테레오 매칭기 생성
    method: 'BM' (Block Matching) 또는 'SGBM' (Semi-Global Block Matching)
    """
    if method == 'BM':
        # Block Matching
        stereo = cv2.StereoBM_create(numDisparities=16*5, blockSize=15)
    else:
        # Semi-Global Block Matching (더 정확하지만 느림)
        stereo = cv2.StereoSGBM_create(
            minDisparity=0,
            numDisparities=16*5,  # 80
            blockSize=5,
            P1=8*3*5**2,  # Smoothness penalty for neighboring pixels
            P2=32*3*5**2,  # Smoothness penalty for larger differences
            disp12MaxDiff=1,
            uniquenessRatio=10,
            speckleWindowSize=100,
            speckleRange=32,
            preFilterCap=63,
            mode=cv2.STEREO_SGBM_MODE_SGBM_3WAY
        )
    return stereo

# SGBM 매칭기 생성 (더 정확함)
stereo_matcher = create_stereo_matcher(method='SGBM')
print("스테레오 매칭기 생성 완료")

In [ ]:
# Cell 4: 스테레오 이미지 로딩 및 전처리
def load_stereo_images(left_path, right_path):
    """왼쪽/오른쪽 스테레오 이미지 로딩"""
    left_img = cv2.imread(str(left_path), cv2.IMREAD_GRAYSCALE)
    right_img = cv2.imread(str(right_path), cv2.IMREAD_GRAYSCALE)
    
    if left_img is None or right_img is None:
        raise ValueError(f"이미지를 로드할 수 없습니다: {left_path}, {right_path}")
    
    return left_img, right_img

# 이미지 로드 (KITTI 형식에 맞게 경로 수정 필요)
# 일반적으로 training/image_02 (left)와 training/image_03 (right) 사용
image_02_dir = project_root / "dataset" / "training" / "image_02"
image_03_dir = project_root / "dataset" / "training" / "image_03"

# 또는 kitti 폴더 내의 이미지 사용
train_images_dir = dataset_root / "images" / "train"

print("이미지 디렉토리 확인:")
print(f"  image_02 (left): {image_02_dir.exists()}")
print(f"  image_03 (right): {image_03_dir.exists()}")
print(f"  kitti images: {train_images_dir.exists()}")

In [ ]:
# Cell 5: 깊이 추정 함수
def compute_depth_map(left_img, right_img, stereo_matcher, calib=None):
    """
    스테레오 이미지로부터 깊이 맵 계산
    
    Args:
        left_img: 왼쪽 이미지 (그레이스케일)
        right_img: 오른쪽 이미지 (그레이스케일)
        stereo_matcher: 스테레오 매칭기
        calib: 칼리브레이션 정보 (P2, P3 포함)
    
    Returns:
        disparity_map: 시차 맵
        depth_map: 깊이 맵
    """
    # 시차 계산
    disparity = stereo_matcher.compute(left_img, right_img).astype(np.float32) / 16.0
    
    # 깊이 계산 (칼리브레이션이 있는 경우)
    if calib and 'P2' in calib and 'P3' in calib:
        # P2, P3에서 baseline과 focal length 추출
        P2 = calib['P2']
        P3 = calib['P3']
        
        # baseline = -P3[0,3] / P3[0,0] - P2[0,3] / P2[0,0]
        focal_length = P2[0, 0]
        baseline = -P3[0, 3] / P3[0, 0]
        
        # depth = (focal_length * baseline) / disparity
        depth_map = np.zeros_like(disparity)
        valid_mask = disparity > 0
        depth_map[valid_mask] = (focal_length * baseline) / disparity[valid_mask]
    else:
        # 칼리브레이션이 없으면 시차만 반환 (상대적 깊이)
        depth_map = disparity
        print("경고: 칼리브레이션 정보가 없어 상대적 깊이만 계산합니다.")
    
    return disparity, depth_map

# 깊이 맵 계산 (테스트)
# left_img, right_img = load_stereo_images(left_path, right_path)
# disparity, depth_map = compute_depth_map(left_img, right_img, stereo_matcher, calib)

In [ ]:
# Cell 6: 깊이 맵 시각화
def visualize_depth_map(image, disparity, depth_map, title="Depth Estimation"):
    """
    깊이 맵 시각화
    """
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # 원본 이미지
    axes[0, 0].imshow(image, cmap='gray')
    axes[0, 0].set_title("원본 이미지", fontsize=12)
    axes[0, 0].axis('off')
    
    # 시차 맵
    axes[0, 1].imshow(disparity, cmap='jet')
    axes[0, 1].set_title("Disparity Map", fontsize=12)
    axes[0, 1].axis('off')
    
    # 깊이 맵 (거리)
    im1 = axes[1, 0].imshow(depth_map, cmap='jet', vmin=0, vmax=depth_map[depth_map > 0].max() if (depth_map > 0).any() else 100)
    axes[1, 0].set_title("Depth Map (meters)", fontsize=12)
    axes[1, 0].axis('off')
    plt.colorbar(im1, ax=axes[1, 0])
    
    # 깊이 맵 히스토그램
    valid_depths = depth_map[depth_map > 0]
    if len(valid_depths) > 0:
        axes[1, 1].hist(valid_depths, bins=50, edgecolor='black')
        axes[1, 1].set_title("Depth Histogram", fontsize=12)
        axes[1, 1].set_xlabel("Depth (meters)")
        axes[1, 1].set_ylabel("Frequency")
    
    plt.suptitle(title, fontsize=14)
    plt.tight_layout()
    plt.show()
    return fig

# 시각화 실행 (예시)
# if 'depth_map' in locals() and 'disparity' in locals():
#     fig = visualize_depth_map(left_img, disparity, depth_map)

In [ ]:
# Cell 7: YOLO 박스 + 깊이 → 3D 위치 추정
def bbox_to_3d(bbox_2d, depth_map, calib, method='center'):
    """
    2D 바운딩 박스와 깊이 맵을 사용하여 3D 위치 추정
    
    Args:
        bbox_2d: [x1, y1, x2, y2] 형태의 바운딩 박스
        depth_map: 깊이 맵
        calib: 칼리브레이션 정보 (P2 포함)
        method: 'center' (중심점) 또는 'median' (중간값)
    
    Returns:
        x, y, z: 3D 좌표 (카메라 좌표계)
    """
    x1, y1, x2, y2 = bbox_2d
    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
    
    # 박스 내부의 깊이 값 추출
    box_depth = depth_map[y1:y2, x1:x2]
    valid_depths = box_depth[box_depth > 0]
    
    if len(valid_depths) == 0:
        return None, None, None
    
    # 깊이 추정 (중심점 또는 중간값)
    if method == 'center':
        center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2
        z = depth_map[center_y, center_x]
    else:  # median
        z = np.median(valid_depths)
    
    if z <= 0:
        return None, None, None
    
    # 칼리브레이션을 사용하여 3D 좌표 계산
    if calib and 'P2' in calib:
        P2 = calib['P2']
        fx, fy = P2[0, 0], P2[1, 1]
        cx, cy = P2[0, 2], P2[1, 2]
        
        # 이미지 좌표 → 카메라 좌표
        u, v = (x1 + x2) // 2, (y1 + y2) // 2
        x = (u - cx) * z / fx
        y = (v - cy) * z / fy
        
        return x, y, z
    
    return None, None, z  # z만 반환

# 테스트
# if 'depth_map' in locals() and 'calib' in locals():
#     test_bbox = [100, 100, 200, 200]  # 예시 바운딩 박스
#     x, y, z = bbox_to_3d(test_bbox, depth_map, calib)
#     print(f"3D 위치: x={x:.2f}m, y={y:.2f}m, z={z:.2f}m")